(Note that all units unless otherwise mentioned are SI units.)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle




ModuleNotFoundError: No module named 'prop'

Importing all the required libraries. 

In [ ]:
zp = np.load('Au_100_zp.npy')
parameters = pickle.load(open('parameters_Au_100.pickle','rb'))
for i in parameters : print(i,parameters[i])
zp_coords = parameters['zp_coords']
plt.imshow(zp,extent=zp_coords)
plt.xlabel('axes in um')
plt.show()

Importing zone plate pattern and the parameters associated with it.

In [ ]:
'''
decide whether to use TF or IR approach depending on the distance
'''
def decide(step_z,step_xy,L,wavel):
    dist = step_z
    sampling = step_xy
    critical = wavel*dist/L
    if sampling > critical :
        p = prop.propTF
        print('propogator to be used : TF')
    else :
        p = prop.propIR
        print('propogator to be used : IR')
    return p   
'''
used as part of the multislice loop
'''
def modify(wavefront,zp_delta,zp_beta,step_z,wavel):
    dist = step_z
    kz = 2 * np.pi * dist /wavel
    beta_slice = zp_beta
    delta_slice = zp_delta
    output = wavefront * np.exp((kz * delta_slice) * 1j) * np.exp(-kz * beta_slice)
    return output
'''
perform free space propogation using the method decided above
'''
def propogate(wavefront,step_xy,step_z,L,wavel,p):
    sampling = step_xy
    dist = step_z
    return p(wavefront,sampling,L,wavel,dist)

*decide* : decides whether TF or IR approach should be used for propogation
* *Inputs* : step size in z, step size in xy, support length, wavelength
* *Outputs* : propogator
    
*modify* : wavefront is modified according to the material present
* *Inputs* : wavefront, slice properties (here the zone plate),step size in z , wavelength
* *Outputs* : modified wavefront

*propogate* : wavefront is propogated for the specified distance
* *Inputs* : wavefront, step size in z, step size in xy, wavelength, propogator
* *Outputs* : wavefront at output plane


In [ ]:
zp_beta  = parameters['beta']*zp
zp_delta = parameters['delta']*zp
step_xy = parameters['step_xy']
energy = parameters['energy(in eV)']
wavel = parameters['wavelength in m']
f = parameters['focal_length']
step_z = 1e-6               
L = step_xy*np.shape(zp)[0] 
n = np.shape(zp)[0]         
zp_thickness = 1000e-9      

*Setting up the parameters for the simulation*<br>
* *zp_beta and zp_delta* : delta and beta grid for the zone plate <br>
* *step_xy*              : sampling in the xy plane<br>
* *wavel*                : wavelength<br>
* *f*                    : focal_length<br>
* *L*                    : support length<br>
* *n*                    : dimension of input zp<br> 
* *zp_thickness*         : thickness of zone plate <br>

In [ ]:
eps1 = 0.1
eps2 = 0.1
delta_z_suggested = ((eps2*(step_xy**2))/(eps1**2*wavel))
number_of_steps_zp = int(np.ceil(zp_thickness/delta_z_suggested))
print('suggested step size :',delta_z_suggested)
print('number of steps required for propogation through the zone plate :',number_of_steps_zp)

Determining the step size in the z direction (direction of propogation) using the meteric described in *Optics Express Vol. 25, Issue 3, pp. 1831-1846*

In [ ]:
theta = 0.05*(np.pi/180)
slope = np.tan(theta)
x = np.linspace(zp_coords[0]*1e-6,zp_coords[1]*1e-6,n)
X,Y = np.meshgrid(x,x)
z1 = 2*np.pi*(1/wavel)*slope*X
wave_in = np.multiply(np.ones((n,n),dtype='complex64'),np.exp(1j*(z1)))

*creating the input wave*<br>
* *theta*   : tilt angle <br>
* *slope*   : tangent of the tilt angle<br>
* *x*       : axis in the x direction<br>
* *X*       : 2D grid containing the x-coordinate of every point on the grid<br>
* *z1*      : phase of the plane wave calculated accoring to the distance traveled<br>
* *wave_in* : input wavefront with phase in accordance with the tilt

In [ ]:
step_z = zp_thickness/number_of_steps_zp
wavefront = np.ones(np.shape(zp),dtype='complex64') #initialize wavefront 
p = decide(step_z,step_xy,L,wavel)
for i in range(number_of_steps_zp):
    wavefront = modify(wave_in,zp_delta,zp_beta,step_z,wavel)
    wavefront  = p(wavefront,step_xy,L,wavel,step_z)
wavefront_exit = wavefront

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)
im1 = ax1.imshow(abs(wavefront_exit),extent = zp_coords)
fig.colorbar(im1,ax = ax1)
ax1.set_xlabel('axes in um')
im2 = ax2.imshow(unwrap_phase(np.angle(wavefront_exit)),extent = zp_coords)
fig.subplots_adjust(right=1.75)
ax2.set_xlabel('axes in um')
fig.colorbar(im2,ax = ax2)
plt.show()
print('magnitude(left) and phase(right) of the exit wave')

In [ ]:
step_z = f
p = decide(step_z,step_xy,L,wavel)
print(step_xy,wavel)
print('Fresnel Number :',((50e-6*50e-6)/(wavel*step_z)))
wavefront_focus  = p(wavefront_exit,step_xy,L,wavel,step_z)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2)
im1 = ax1.imshow(np.log(abs(wavefront_focus)),extent = zp_coords)
fig.colorbar(im1,ax = ax1)
ax1.set_xlabel('axes in um')
im2 = ax2.imshow(unwrap_phase(np.angle(wavefront_focus)),extent = zp_coords)
fig.subplots_adjust(right=1.75)
ax2.set_xlabel('axes in um')
fig.colorbar(im2,ax = ax2)
plt.show()
print('magnitude(in log scale)(left) and phase(right) of the wave in the focal plane')
print('maximum value of wavefront in focal plane : ',np.max(abs(wavefront_focus)))
print('minimum value of wavefront in focal plane : ',np.min(abs(wavefront_focus)))
print('location of maxima in focal plane : ',np.where(abs(wavefront_focus)==np.max(abs(wavefront_focus))))

In [ ]:
x = np.linspace(zp_coords[0],zp_coords[1],n)
plt.plot(x,abs(wavefront_focus)[5000,:])
plt.xlabel('axes in um')
plt.show()
print('slice in x-direction along the focal spot')